In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
local=False
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .appName("aida_poc_etl").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("aida_poc_etl") \
                      .config("spark.kubernetes.container.image",os.environ["IMAGE_NAME"]) \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "10") \
                      .config("spark.executor.memory","16g") \
                      .config("spark.driver.memory","32g") \
                      .config('spark.jars.packages','org.postgresql:postgresql:42.2.24') \
                      .enableHiveSupport() \
                      .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f3071806-ba43-4163-826c-9ebf5aa0254a;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.24 in central
	found org.checkerframework#checker-qual;3.5.0 in central
downloading https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.24/postgresql-42.2.24.jar ...
	[SUCCESSFUL ] org.postgresql#postgresql;42.2.24!postgresql.jar (113ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.5.0/checker-qual-3.5.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.5.0!checker-qual.jar (25ms)
:: resolution report :: resolve 462ms :: artifacts dl 142ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.2.24 from central in [default]
	-------------------------------

In [3]:
work_dir="s3a://projet-poc-aida/rp"
parquet_file_name="individus_snappy_parquet"
data_path=f"{work_dir}/{parquet_file_name}"

# Step 1: Prepare source dataframe

Use spark context to read a parquet file and return a data frame 

In [4]:
df_parquet=spark.read.parquet(data_path)

In [11]:
df_parquet.show(5)

+----------------+---------------------+-----------------+--------------+-------------------+---------------+------------------+-------------+--------------+------------------+----+----------+-------+----------------+-------+---+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|region_residence|departement_residence|commune_residence|region_travail|departement_travail|commune_travail|commune_anterieure|commune_etude|pays_naissance|             poids|sexe|statut_pro|densite|recherche_emplo

# Step2: Create a table in hive metastore

Use the spark dataframe to create a hive table in the hive metastore. So we can reuse it for later. 


In [6]:
table_name="individus_test"

In [7]:
schema_str = ', '.join([' '.join(x) for x in df_parquet.dtypes])

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {table_name}
({schema_str})
STORED as parquet LOCATION '{data_path}'
""")


2022-05-30 13:47:32,590 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [8]:
spark.sql('show tables;').show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|  default|     individus|      false|
|  default|individus_test|      false|
+---------+--------------+-----------+



Now your hive table has been created. In the backgroud, if you enabled the listener, the metadata of this hive table will be uploaded to our [data catalog](https://atlas.lab.sspcloud.fr/index.html#!/search). So you can find all your hive table easily even you don't have notebook anymore.

You can try to use the search engine of our [data catalog](https://atlas.lab.sspcloud.fr/index.html#!/search) to find your table. 

In [9]:
spark.sql(f"""SELECT * FROM {table_name} limit 5""").show()

+----------------+---------------------+-----------------+--------------+-------------------+---------------+------------------+-------------+--------------+------------------+----+----------+-------+----------------+-------+---+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|region_residence|departement_residence|commune_residence|region_travail|departement_travail|commune_travail|commune_anterieure|commune_etude|pays_naissance|             poids|sexe|statut_pro|densite|recherche_emplo

# Step 3. Drop table 

You can delete your table if you don't need it anymore. You will notice the metadata of the deleted table are `removed` from the data catalog too.  

In [10]:
spark.sql(f"""drop table if exists {table_name}""").show()
spark.sql('show tables;').show()

++
||
++
++

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|individus|      false|
+---------+---------+-----------+

